Importe les packages pour le notebook

In [1]:
%%capture
!pip3 install rdflib pandas

Importe les librairies python

In [2]:
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import XSD
import json
import datetime

Encode en utf-8 et replis les cellules vide par des chaine de caracteres vides

In [3]:
url='https://www.data.gouv.fr/fr/datasets/r/64e02cff-9e53-4cb2-adfd-5fcc88b2dc09'
# encode les valeurs en utf-8
df=pd.read_csv(url,sep=";", dtype={'prix_id': str, 'com_arm_code': str, 'epci_code': str, 'dep_code': str, 'com_code': str, 'cp': str}, encoding='utf-8')
# remplace les cellule vide du csv par des chaines vides
df = df.fillna('')

Affiche le csv

In [4]:
df

,id,cp,pop,adresse,ville,horaires,geom,prix_maj,prix_id,prix_valeur,...,epci_code,epci_name,dep_code,dep_name,reg_code,reg_name,com_code,com_name,services_service,horaires_automate_24_24
0,33480002,33480,R,55 AVENUE GAMBETTA,CASTELNAU-DE-MéDOC,,"45.033,-0.799",2023-05-12T12:19:23+02:00,1,1.752,...,243301389,CC Médullienne,33,Gironde,75.0,Nouvelle-Aquitaine,33104,Castelnau-de-Médoc,Toilettes publiques//Boutique non alimentaire/...,Non
1,91940008,91940,R,1 AV DE L OCEANIE ZAC COURTABOEU,LES ULIS,"{""@automate-24-24"": ""1"", ""jour"": [{""@id"": ""1"",...","48.678,2.199",2023-06-06T02:01:00+02:00,6,1.915,...,200056232,CA Communauté Paris-Saclay,91,Essonne,11.0,Île-de-France,91692,Les Ulis,Boutique alimentaire//Boutique non alimentaire...,Oui
2,86000023,86000,R,10 Boulevard Jeanne d'Arc,Poitiers,"{""@automate-24-24"": """", ""jour"": [{""@id"": ""1"", ...","46.589,0.34",2023-06-06T08:54:16+02:00,1,1.856,...,200069854,CU du Grand Poitiers,86,Vienne,75.0,Nouvelle-Aquitaine,86194,Poitiers,Relais colis//Boutique alimentaire//Boutique n...,Non
3,13270004,13270,R,DEVIATION RN 568 OUEST,Fos-sur-Mer,"{""@automate-24-24"": """", ""jour"": [{""@id"": ""1"", ...","43.441,4.94",2023-06-06T06:01:00+02:00,6,1.891,...,200054807,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur,13039,Fos-sur-Mer,,Non
4,13380002,13380,R,89 AVENUE DE LA LIBERATION,Plan-de-Cuques,"{""@automate-24-24"": """", ""jour"": [{""@id"": ""1"", ...","43.34118223755699,5.456789325646399",2023-06-06T13:08:00+02:00,3,1.108,...,200054807,Métropole d'Aix-Marseille-Provence,13,Bouches-du-Rhône,93.0,Provence-Alpes-Côte d'Azur,13075,Plan-de-Cuques,,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33295,71000001,71000,R,RN 6,MâCON,"{""@automate-24-24"": """", ""jour"": [{""@id"": ""1"", ...","46.32536,4.83854",2023-06-06T13:37:02+02:00,4,0.879,...,200070308,CA Mâconnais Beaujolais Agglomération,71,Saône-et-Loire,27.0,Bourgogne-Franche-Comté,71270,Mâcon,Toilettes publiques//Laverie//Relais colis//Bo...,Non
33296,61200008,61200,A,Lieu-dit Les Fermes,Fontenai-sur-Orne,"{""@automate-24-24"": """", ""jour"": [{""@id"": ""1"", ...","48.722216324002,-0.056921417848999",2023-05-24T00:00:00+02:00,1,1.92,...,200068450,CC Argentan Intercom,61,Orne,28.0,Normandie,61153,Écouché-les-Vallées,Toilettes publiques//Douches//Boutique aliment...,Non
33297,31530003,31530,R,ZA DE LARCENNE,SAINT-PAUL-SUR-SAVE,"{""@automate-24-24"": ""1"", ""jour"": [{""@id"": ""1"",...","43.698,1.219",2023-06-05T10:59:25+02:00,5,1.769,...,200071314,CC des Hauts Tolosans,31,Haute-Garonne,76.0,Occitanie,31507,Saint-Paul-sur-Save,Piste poids lourds//Lavage automatique//Lavage...,Oui
33298,17137001,17137,R,44 Avenue de la Rochelle,NIEUL-SUR-MER,"{""@automate-24-24"": ""1"", ""jour"": [{""@id"": ""1"",...","46.198,-1.158",2023-05-31T10:23:07+02:00,1,1.792,...,241700434,CA de La Rochelle,17,Charente-Maritime,75.0,Nouvelle-Aquitaine,17264,Nieul-sur-Mer,Carburant additivé//Services réparation / entr...,Oui


Déclaration des namespaces 

In [5]:
ns = Namespace("http://example.org/data/")
gasStationNS = ns.GasStation
epciNS = ns.epci
offerCatalogNS = ns.OfferCatalog
SCHEMA = Namespace('http://schema.org/')
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")

Fonction pour transformer le format des dates

In [6]:
def formater_heure(heure_str):
    heure_obj = datetime.datetime.strptime(heure_str, "%H.%M").time()
    return heure_obj.strftime("%H:%M")

Creation du graphe et transformation des informations en RDF

In [7]:
g = Graph()

for index, row in df.iterrows():
  station = URIRef(gasStationNS + "/" + Literal(row['id']))

  if not (station, RDF.type, SCHEMA.GasStation) in g:
    address = URIRef(station + "/address")
    epci = URIRef(epciNS + "/" + Literal(row['epci_code']));
    offerCatalog = URIRef(offerCatalogNS + "/" + Literal(row['epci_code']));

    g.add((station, RDF.type, SCHEMA.GasStation))

    g.add((station, SCHEMA.hasOfferCatalog, offerCatalog))
    g.add((offerCatalog, RDF.type, SCHEMA.OfferCatalog))
    g.add((offerCatalog, SCHEMA.itemListElement, Literal(row['services_service'])))

    daysMap = {'Lundi' : 'Mo', 'Mardi' : 'Tu', 'Mercredi' : 'We', 'Jeudi' : 'Th', 'Vendredi' : 'Fr', 'Samedi' : 'Sa', 'Dimanche' : 'Su'}
    if (row['horaires'] != ''):
      openingHoursTab = {}
      for jour in json.loads(row['horaires'])['jour']:
        if (jour['@ferme'] != 1 and "horaire" in jour):
          day = daysMap[jour['@nom']]
          spec_uri = URIRef(station + '/' + day.lower())
          if not isinstance(jour['horaire'], list):
            jour['horaire'] = [jour['horaire']]
          for horaire in jour['horaire']:
            hours = formater_heure(horaire['@ouverture']) + '-' + formater_heure(horaire['@fermeture'])
            if not hours in openingHoursTab:
              openingHoursTab[hours] = []
            if not day in openingHoursTab[hours]:
              openingHoursTab[hours].append(day)
      for oh in openingHoursTab:
        line = ','.join(openingHoursTab[oh]) + ' ' + oh
        g.add((station, SCHEMA.openingHours, Literal(line)))


    pointP = row['geom'].split(",")
    g.add((station, SCHEMA.longitude, Literal(pointP[0], datatype=XSD.float)))
    g.add((station, SCHEMA.latitude, Literal(pointP[1], datatype=XSD.float)))

    g.add((station, SCHEMA.address, address))
    g.add((address, RDF.type, SCHEMA.PostalAddress))
    g.add((address, SCHEMA.postalCode, Literal(row['cp'])))
    g.add((address, SCHEMA.streetAddress, Literal(row['adresse'])))
    g.add((address, SCHEMA.addressLocality, Literal(row['ville'])))
    g.add((address, SCHEMA.addressRegion, Literal(row['reg_name'])))

    if not (epci, RDF.type, SCHEMA.Organization) in g:
      g.add((epci, RDF.type, SCHEMA.Organization))
      g.add((epci, SCHEMA.name, Literal(row['epci_name'])))
      
    g.add((epci, SCHEMA.member, station))
    g.add((station, SCHEMA.memberOf, epci))

  if row['prix_id'] != '':
    gas = URIRef(station + '/gas/' + str(row['prix_id']))
    g.add((station, SCHEMA.makesOffer, gas))
    g.add((gas, RDF.type, SCHEMA.Offer))
    g.add((gas, SCHEMA.name, Literal(row['prix_nom'])))
    g.add((gas, SCHEMA.price, Literal(row['prix_valeur'], datatype=XSD.float)))

Serialisation du graphe en turtle

In [8]:
print(g.serialize(destination="stations.ttl", format='turtle'))

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].
